In [ ]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.



Problem Statement - A sample life insurance policy is shared with us. Our goal here is to build a simple RAG application on the policy statement pdf document.

Solution Strategy - Build a POC/MVP which should solve the following requirements:

Users would get responses from the policy document.
If they want to refer to the original page from which the bot is responding, the bot should provide a citation as well.

Goal - Solving the above two requirements well in the POC/MVP would ensure that the accuracy of the overall model is good and therefore further improvisations and customizations make sense.

Data Used - Principal Sample Life Insurance documents stored in a single folder

Tools used - LlamaIndex (only for now) has been used due to its powerful query engine, fast data processing using data loaders and directory readers as well as easier and faster implementation using fewer lines of code.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Example: If the file is in a different directory, provide the full path
with open('/content/drive/MyDrive/LLaMa_Index_Session_CollabNotebook/api_key.txt', 'r') as key_file:
    OPENAI_API_KEY = key_file.read().strip()

In [ ]:
#Loading docx2txt for document reading related dependencies
!pip install docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=4df6599cb7c7c23b3d3d0dbd8246431a75d114d2b07d6416627a6d1e2cec2cb8
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install openai

In [ ]:
pip install llama_index[langchain]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.7 MB/s eta 0:00:00


In [ ]:
from llama_index.llms import ChatMessage, OpenAI
import os
import openai

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set the API key
filepath = "/content/drive/MyDrive/LLaMa_Index_Session_CollabNotebook/"
with open(filepath + "api_key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [ ]:
from llama_index import SimpleDirectoryReader
reader = SimpleDirectoryReader(input_dir="/content/drive/MyDrive/LLaMa_Index_Session_CollabNotebook/Semantic_spotter")

In [ ]:
?SimpleDirectoryReader

In [ ]:
documents = reader.load_data()
print(f"Loaded {len(documents)} docs")

Loaded 64 docs


In [ ]:
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex
from IPython.display import display, HTML

# create parser and parse document into nodes
parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(documents)

# # build index
index = VectorStoreIndex(nodes)

# Construct Query Engine
query_engine = index.as_query_engine(similarity_top_k = 4)

In [ ]:
response = query_engine.query("What is the premium for life insurance?")

In [ ]:
#Checking the response
response.response

'The premium for life insurance is $0.210 for each $1,000 of insurance in force.'

In [ ]:
#Check the source node
response.source_nodes[1]

NodeWithScore(node=TextNode(id_='9b249f6c-d0ab-48b6-a321-fc774df17ab8', embedding=None, metadata={'page_label': '20', 'file_name': 'Principal-Sample-Life-Insurance-Policy.pdf', 'file_path': '/content/drive/MyDrive/LLaMa_Index_Session_CollabNotebook/Semantic_spotter/Principal-Sample-Life-Insurance-Policy.pdf', 'file_type': 'application/pdf', 'file_size': 222772, 'creation_date': '2024-02-10', 'last_modified_date': '2024-01-13', 'last_accessed_date': '2024-02-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2aa12083-ff9b-484a-af30-c419176d7b63', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '20', 'file_name': 'Principal-Sample-Life-Insurance-Policy.pdf', 'file_path': '/content/drive/MyDri

In [ ]:
#Extract the file name and page number
response.source_nodes[1].node.metadata['file_name'] + " page no " + response.source_nodes[1].node.metadata['page_label']

'Principal-Sample-Life-Insurance-Policy.pdf page no 20'

In [ ]:
#Extract the score
response.source_nodes[1].score

0.8401891601555638

In [ ]:
# Response node text
response.source_nodes[1].node.text

'This policy has been updated effective  January 1, 2014  \n \n \nPART II - POLICY ADMINISTRATION  \nGC 6004  Section B - Premiums, Page 1  \n Section B -  Premiums  \n \n Article 1 - Payment Responsibility; Due Dates; Grace Period  \n \nThe Policyholder is responsible for collection and payment of all premiums due while this Group \nPolicy is in force.  Payments must be sent to the home office of The Principal in Des Moines, Iowa.  \n The first premium is due on the Date of Issue of this Group Policy.  Each premium thereafter will be due on the first of each Insurance Month.  Except for the first premium, a Grace Period of \n31 days will be allowed for payment of premium.  "Grace Period" means the first 31- day period \nfollowing a premium due date.  The Group Policy will remain in fo rce until the end of the Grace \nPeriod, unless the Group Policy has been terminated by notice as described in PART II, Section C.  The Policyholder will be liable for payment of the premium for the time

In [ ]:
## Query response function
def query_response(user_input):
  response = query_engine.query(user_input)
  file_name = response.source_nodes[0].node.metadata['file_name'] + " page nos " + response.source_nodes[0].node.metadata['page_label'] + "," + response.source_nodes[1].node.metadata['page_label']
  final_response = response.response + '\n Check further at ' + file_name
  return final_response

In [ ]:
def initialize_conv():
  print('Feel free to ask Questions regarding the annual budget of 2024-25 Press exit once you are done')
  while True:
    user_input = input()
    if user_input.lower() == 'exit':
      print('Exiting the program... bye')
      break
    else:
      response = query_response(user_input)
      display(HTML(f'<p style="font-size:20px">{response}</p>'))

In [ ]:
initialize_conv()

Feel free to ask Questions regarding the annual budget of 2024-25 Press exit once you are done
What is the premium for life insurance?


What is the premium for accidental death to be covered?


What is the grace period for the insurance paying term allowed?


What is the benefit for members with accidental death ?


exit
Exiting the program... bye


In [ ]:
questions = [
    'What is the premium for life insurance?',
    'What is the premium for accidental death to be covered?',
    'What is the grace period for the insurance paying term allowed?',
    'What is the benefit for members with accidental death?'
]


In [ ]:
def testing_pipeline(questions):
  test_feedback  = []
  for i in questions:
    print(i)
    print(query_response(i))
    print('\n Please provide your feedback on the response provided by the bot')
    user_input = input()
    page = query_response(i).split()[-1]
    test_feedback.append((i,query_response(i),page,user_input))

  feedback_df = pd.DataFrame(test_feedback, columns =['Question', 'Response', 'Page','Good or Bad'])
  return feedback_df

In [ ]:
import pandas as pd

In [ ]:
testing_pipeline(questions)

What is the premium for life insurance?
The premium for life insurance is $0.210 for each $1,000 of insurance in force.
 Check further at Principal-Sample-Life-Insurance-Policy.pdf page nos 21,20

 Please provide your feedback on the response provided by the bot
Good
What is the premium for accidental death to be covered?
I'm sorry, but the context information provided does not mention anything about the premium for accidental death coverage.
 Check further at Principal-Sample-Life-Insurance-Policy.pdf page nos 54,53

 Please provide your feedback on the response provided by the bot
Bad
What is the grace period for the insurance paying term allowed?
The grace period for the insurance paying term allowed is 31 days.
 Check further at Principal-Sample-Life-Insurance-Policy.pdf page nos 23,20

 Please provide your feedback on the response provided by the bot
Good
What is the benefit for members with accidental death?
The benefit for members with accidental death is 100% of the Scheduled B

,Question,Response,Page,Good or Bad
0,What is the premium for life insurance?,The premium for life insurance is $0.210 for e...,"21,20",Good
1,What is the premium for accidental death to be...,The context information does not provide any i...,"54,53",Bad
2,What is the grace period for the insurance pay...,The grace period for the insurance paying term...,"23,20",Good
3,What is the benefit for members with accidenta...,The benefit for members with accidental death ...,"54,55",Good


In [ ]:
response = query_engine.query("What is the premium for accidental death to be covered?")

In [ ]:
response.response

'The context information does not provide any information about the premium for accidental death coverage.'

In [ ]:
response.source_nodes[3]

NodeWithScore(node=TextNode(id_='31ba54bd-4229-47cf-8b31-6c49571fd3ce', embedding=None, metadata={'page_label': '55', 'file_name': 'Principal-Sample-Life-Insurance-Policy.pdf', 'file_path': '/content/drive/MyDrive/LLaMa_Index_Session_CollabNotebook/Semantic_spotter/Principal-Sample-Life-Insurance-Policy.pdf', 'file_type': 'application/pdf', 'file_size': 222772, 'creation_date': '2024-02-10', 'last_modified_date': '2024-01-13', 'last_accessed_date': '2024-02-10'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='42acaff5-9f40-43df-9172-a7a4322a8899', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '55', 'file_name': 'Principal-Sample-Life-Insurance-Policy.pdf', 'file_path': '/content/drive/MyDri

In [ ]:
response.source_nodes[1].node.text

"This policy has been updated effective  January 1, 2014  \n \n      PART IV - BENEFITS  \nGC 6015  Section B - Member Accidental Death and \nDismemberment Insurance, Page 1  \n Section B -  Member Accidental Death and Dismemberment Insurance  \n \n Article 1 - Schedule of Insurance  \n \nSubject to the Effective Date provisions of PART III, Section B, and the qualifying provisions of \nthis Section B, the Scheduled Benefit for an insured Member will be based on his or her class:  \n \nClass  *Scheduled Benefit  \n  \nALL MEMBERS  $10,000  \n  *The Scheduled Benefit is subject to the Proof of Good Health requirements as shown in PART III, Section B, Article 1.  Because of the Proof of Good Health requirements, the amount of insurance approved by The Principal may be different than the Scheduled Benefit.  If the approved amount of insurance is different than the Scheduled Benefit, the approved amount will \napply.  \n \nFor the age(s) shown below, the amount of a Member's insurance will

In [ ]:
retrieved = response.source_nodes[0].node.text + response.source_nodes[1].node.text + response.source_nodes[2].node.text + response.source_nodes[3].node.text

In [ ]:
messages = [
    {"role":"system", "content":"You are an AI assistant to user."},
    {"role":"user", "content":f"""What is the premium for accidental death to be covered? Check in '{retrieved}' """},
          ]

In [ ]:
response2 = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages)
response2.choices[0].message.content

'The premium for accidental death coverage is not provided in the information you provided. It only outlines the benefits that will be payable in the event of accidental death or dismemberment. To determine the premium, you would need to contact the insurance provider or refer to the insurance policy documents.'

In [ ]:
# Import necessary libraries and modules
from llama_index import ServiceContext, LLMPredictor, OpenAIEmbedding, PromptHelper
from llama_index.llms import OpenAI
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser

# Initialize an LLMPredictor object with a specific OpenAI model and settings
llm = OpenAI(model='gpt-3.5-turbo', temperature=0, max_tokens=256)

# Initialize an OpenAIEmbedding model
embed_model = OpenAIEmbedding()

# Initialize a SimpleNodeParser with custom chunking settings
node_parser = SimpleNodeParser.from_defaults(chunk_size=2000, chunk_overlap=50)

# Initialize a PromptHelper with specific context and output settings
prompt_helper = PromptHelper(
  context_window=4096,
  num_output=512,
  chunk_overlap_ratio=0.1,
)

# Create a ServiceContext by combining the above components
service_context = ServiceContext.from_defaults(
  llm=llm,
  embed_model=embed_model,
  node_parser=node_parser,
  prompt_helper=prompt_helper
)

# Create a VectorStoreIndex from a list of documents using the service context
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

# Initialize a query engine for the index with a specified similarity top-k value
query_engine = index.as_query_engine(similarity_top_k=3)

# Query the engine with a specific question
response = query_engine.query("What is the premium for accidental death to be covered?")

# Display the synthesized response with HTML formatting
display(HTML(f'<p style="font-size:20px">{response.response}</p>'))